In [1]:
import requests
import pandas as pd

import tqdm

import random
import time

from bs4 import BeautifulSoup

from os import path

In [2]:
tipo_imovel = "apartamentos"

In [3]:
if path.exists(f"data/{tipo_imovel}_completo_intermediario.csv"):
    df = pd.read_csv(f"data/{tipo_imovel}_completo_intermediario.csv")
else:
    df = pd.read_csv(f"data/{tipo_imovel}_incompleto.csv")
    df['condominio'] = ''
    df['IPTU'] = ''
    df['lat'] = ''
    df['long'] = ''
    df['area_total'] = ''
    df['area_util'] = ''
    df['banheiros'] = ''
    df['vagas'] = ''
    df['quartos'] = ''
    df['suites'] = ''
    df['idade'] = ''
    df['endereco'] = ''
    df['bairro'] = ''
    df['idade_anuncio'] = ''
    df['downloaded'] = False

print(df.shape)
df.head()

(25902, 18)


,nome,valor_venda,url,downloaded,condominio,IPTU,lat,long,area_total,area_util,banheiros,vagas,quartos,suites,idade,endereco,bairro,idade_anuncio
0,Sqn 107 Asa Norte Apartamento 4 Quartos 1 Suít...,1390000,https://www.wimoveis.com.br/propriedades/sqn-1...,True,R$ 970,NaN,-15.790497,-47.884679,169m²,133m²,3.0,1.0,4.0,1.0,NaN,"QD Sqn 107 , Asa Norte, Brasília",", Asa Norte, Brasília",mais de 1 ano\n
1,Frente Ao Parque Burle Marx - 4 Súite - Nw 110,2108549,https://www.wimoveis.com.br/propriedades/frent...,True,NaN,NaN,-15.742130,-47.910920,193m²,192m²,5.0,3.0,4.0,4.0,NaN,"Quadra SQNW 110, Noroeste, Brasília",", Noroeste, Brasília",234
2,Lindo Apto 2 Qtos Decorado Ar - Condicionado A...,350000,https://www.wimoveis.com.br/propriedades/lindo...,True,NaN,NaN,-15.835330,-48.013984,51m²,51m²,1.0,1.0,2.0,0.0,5,"Rua 07 norte, Aguas Norte, Águas Claras",", Aguas Norte, Águas Claras",14
3,"Apartamento à Venda no Guará, 2 Quartos - Resi...",347000,https://www.wimoveis.com.br/propriedades/apart...,True,NaN,NaN,-15.850292,-47.971731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sqn 105 Bloco J 3º Andar, Permuta Por Apto 3q ...",865000,https://www.wimoveis.com.br/propriedades/sqn-1...,True,R$ 700,R$ 1.330,-15.773160,-47.882342,125m²,125m²,3.0,0.0,3.0,1.0,37,"SQN 105 Bloco J, Asa Norte, Asa Norte, Brasília",", Asa Norte, Brasília",71


In [4]:
SAVE_EVERY_N = 100

In [5]:
for i, row in tqdm.tqdm_notebook(df.iterrows(), total=df.shape[0]):
    
    if(row.downloaded == True):
        print(f"item {row.url} ja baixado, passando")
        continue
    
    try:
        
        tqdm.tqdm.write(str(i + 1) +" Buscando item " + row.url)
        response = requests.get(row.url)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Condominio e IPTU
        valores = soup.findAll("div",{"class":"block-expensas block-row"})
        if(len(valores)>0):
            for item in valores:
                #tqdm.tqdm.write(str(item))
                if("Condomínio" in str(item) ):
                    #print(item.find("span").text)
                    df.at[i,'condominio'] = item.find("span").text
                else:
                    df.at[i,'IPTU'] = item.find("span").text
                    
        # latlong
        google_maps_url = soup.find("img", {"class":"static-map"})['src']
        
        lat_long = google_maps_url.split("?center=")[1].split("&zoom=")[0]
        
        lat = lat_long.split(",")[0]
        df.at[i,'lat'] = lat
        
        long = lat_long.split(",")[1]
        df.at[i,'long'] = long
        
        # icones
        icon_total = soup.find("i",{"class":"icon-f icon-f-stotal"})
        if icon_total is not None:
            area_total = icon_total.findNext('b').text
            df.at[i,'area_total'] = area_total
        
        icon_util = soup.find("i",{"class":"icon-f icon-f-scubierta"})
        if icon_util is not None:
            area_util = icon_util.findNext('b').text
            df.at[i,'area_util'] = area_util
            
        icon_banheiro = soup.find("i",{"class":"icon-f icon-f-bano"})
        if icon_banheiro is not None:
            qtd_banheiros = icon_banheiro.findNext('b').text
            df.at[i,'banheiros'] = qtd_banheiros
            
        icon_vagas = soup.find("i",{"class":"icon-f icon-f-cochera"})
        if icon_vagas is not None:
            qtd_vagas = icon_vagas.findNext('b').text
            df.at[i,'vagas'] = qtd_vagas
        
        icon_quartos = soup.find("i",{"class":"icon-f icon-f-dormitorio"})
        if icon_quartos is not None:
            qtd_quartos = icon_quartos.findNext('b').text
            df.at[i,'quartos'] = qtd_quartos
        
        icon_suites = soup.find("i",{"class":"icon-f icon-f-toilete"})
        if icon_suites is not None:
            qtd_suites = icon_suites.findNext('b').text
            df.at[i,'suites'] = qtd_suites
        
        icon_idade = soup.find("i",{"class":"icon-f icon-f-antiguedad"})
        if icon_idade is not None:
            idade = icon_idade.findNext('b').text
            df.at[i,'idade'] = idade
        
        h2_endereco = soup.find("h2",{"class":"title-location"})
        if h2_endereco is not None:
            df.at[i,'endereco'] = h2_endereco.text
            df.at[i,'bairro'] = h2_endereco.findNext("span").text
        
        h5_idade = soup.find("h5",{"class":"section-date css-float-r"})
        if h5_idade is not None:
            df.at[i,'idade_anuncio'] = h5_idade.text.replace("\n Publicado há ","").replace(" dias\n","")
        
    except Exception as e:
        print("exceção ao buscar dados. Mensagem:", str(e))
        pass
    
    df.at[i,'downloaded'] = True
    
    if (i % SAVE_EVERY_N) == 0:
        print(f"Salvando arquivo intermediario em data/{tipo_imovel}_completo_intermediario.csv")
        df.to_csv(f"data/{tipo_imovel}_completo_intermediario.csv", index=False)
    
    #dorme
    aleatorio = random.randint(1, 4)
    tqdm.tqdm.write("agora vou aguardar: " + str(aleatorio))
    time.sleep(aleatorio)

print(df.shape)
df.head(10)

/home/chris/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


item https://www.wimoveis.com.br/propriedades/sqn-107-asa-norte-apartamento-4-quartos-1-suite-2941051941.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/frente-ao-parque-burle-marx-4-suite-nw-110-2946066746.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/lindo-apto-2-qtos-decorado-ar-condicionado-armarios-2949360434.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-no-guara-2-quartos-residencial-2948837864.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqn-105-bloco-j-3-andar-permuta-por-apto-3q-asa-2948712319.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/excelente-apartamento-cobertura-223-m-sup2--vazado.-2949173936.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-3-quartos-com-suite-reformado-res-bella-2949522842.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-no-setor-progre

item https://www.wimoveis.com.br/propriedades/vila-grandino-3-quartos-96-m-2-vagas-refomado-2949441645.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/belissimo-apartamento-no-condominio-via-maxime-com-03-2948752594.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/excelente-vazado-com-03-quartos-na-sqs-410-bloco-e-a-2943768387.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/rua-03-norte-res-piazza-di-spagna-mobiliado-01-2943811488.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/av-araucarias-res-blend-varanda-01-quarto-2949038733.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/eliane-fleury-2-quartos-nascente-63-m-sup2--armarios-2949134979.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/imprensa-iv-03-quartos-a-venda-2948549397.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-com-03-quartos-sala-cozinha-dce-e-2

item https://www.wimoveis.com.br/propriedades/asa-norte-3-suites-c-127-m-sup2--vazado-2-vagas-2948803964.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-no-jardim-goias-2947750782.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/tagua-life-50-m-sup2--1-qto-podendo-transformar-em-2-2948470771.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/gran-reserva-biografia-pronto-sqnw-108-4-suites-2949247679.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/cobertura-400-m-sup2--canto-vazada-piscina-e-sauna-2949258373.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/imperdivel-sqnw-306-128-m-sup2--3-suites-2-2947526888.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-sul-aguas-claras-2946794938.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/3-quartos-vazados-no-noroeste-excelente-planta-vagas-2946963733.h

item https://www.wimoveis.com.br/propriedades/canto-vazado-nascente-vista-livre-3-suites-166-2949461560.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/nascente-vista-livre-lazer-ed-moove-residence-2948163745.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/scrn-716-kitnete-bem-dividida-2944275937.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/orion-setor-central-gama-df-2949470141.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/lindo-apartamento-na-quadra-17-com-elevador-e-garagem-2947805553.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/apartamento-sudoeste-2944207878.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/sqs-404-bloco-o-sqs-404.-ed.-via-omega.-111239312.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/rua-37-real-celebration-2932725735.html ja baixado, passando
item https://www.wimoveis.com.br/propriedades/r

...

agora vou aguardar: 4
25710 Buscando item https://www.wimoveis.com.br/propriedades/apartamentos-a-venda-1-2-quartos-e-loja-comercial-2938988272.html
agora vou aguardar: 3
25711 Buscando item https://www.wimoveis.com.br/propriedades/kit-net-a-venda-condominio-jardim-europa-ii-grande-2949100628.html
agora vou aguardar: 3
25712 Buscando item https://www.wimoveis.com.br/propriedades/sqnw-107-panoramica-pk-burle-marx-99126-9022-2945213585.html
agora vou aguardar: 4
25713 Buscando item https://www.wimoveis.com.br/propriedades/sqs-414-apartamento-com-3-dormitorios-vazado-de-canto-2946096612.html
agora vou aguardar: 2
25714 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-no-noroeste-2-e-3-quartos-2948160963.html
agora vou aguardar: 2
25715 Buscando item https://www.wimoveis.com.br/propriedades/sqn-210-desocupado-com-reformas-venha-conferir-essa-2949272243.html
agora vou aguardar: 2
25716 Buscando item https://www.wimoveis.com.br/propriedades/sqn-213-bloco-k-via-versa

agora vou aguardar: 2
25766 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-4-quartos-suites-duplex-2941490529.html
agora vou aguardar: 3
25767 Buscando item https://www.wimoveis.com.br/propriedades/sqs-207-oportunidade-4-quartos-pronto-para-reforma-2949472734.html
agora vou aguardar: 3
25768 Buscando item https://www.wimoveis.com.br/propriedades/smas-duplex-c-75-m-sup2--podendo-fazer-um-2-suites-2943983821.html
agora vou aguardar: 3
25769 Buscando item https://www.wimoveis.com.br/propriedades/residencial-we-life-2-qtos-andar-alto-nascente-varanda-2946004589.html
agora vou aguardar: 2
25770 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-em-aguas-claras-3-quartos-2948837658.html
agora vou aguardar: 2
25771 Buscando item https://www.wimoveis.com.br/propriedades/sqsw-305-luxuosa-cobertura!-2941420221.html
agora vou aguardar: 1
25772 Buscando item https://www.wimoveis.com.br/propriedades/via-naturale-118-m-sup2--03-quartos-2949014224.h

25822 Buscando item https://www.wimoveis.com.br/propriedades/apartamentos-a-venda-2-e-3-quartos-park-sul-brasilia-2944080228.html
agora vou aguardar: 1
25823 Buscando item https://www.wimoveis.com.br/propriedades/excellence-park-sul-agio-4qts-98243-8383-2949181452.html
agora vou aguardar: 2
25824 Buscando item https://www.wimoveis.com.br/propriedades/perto-do-metro-reformado-central-park-2944284179.html
agora vou aguardar: 4
25825 Buscando item https://www.wimoveis.com.br/propriedades/sqs-107-4-qts-com-264-m-sup2--garagem-coberta-e-2949057305.html
agora vou aguardar: 3
25826 Buscando item https://www.wimoveis.com.br/propriedades/cobertura-a-venda-no-311-by-lotus-2947544640.html
agora vou aguardar: 2
25827 Buscando item https://www.wimoveis.com.br/propriedades/residencial-citta-rua-13-norte-2948944104.html
agora vou aguardar: 2
25828 Buscando item https://www.wimoveis.com.br/propriedades/ae-04-sports-club-residence-belissimo-apartamento-2949472739.html
agora vou aguardar: 1
25829 Buscan

25878 Buscando item https://www.wimoveis.com.br/propriedades/apartamento-a-venda-em-aguas-claras-3-quartos-2948837658.html
agora vou aguardar: 3
25879 Buscando item https://www.wimoveis.com.br/propriedades/aguas-claras-2-quartos-com-suite-rua-25-norte-2949453471.html
agora vou aguardar: 3
25880 Buscando item https://www.wimoveis.com.br/propriedades/oportunidade-lindo-3-qts-nascente-de-canto-2949390156.html
agora vou aguardar: 4
25881 Buscando item https://www.wimoveis.com.br/propriedades/sqn-214-90-m-sup2--canto-garagem-2947365394.html
agora vou aguardar: 1
25882 Buscando item https://www.wimoveis.com.br/propriedades/apartamentos-2-quartos-sendo-1-suite-a-venda-no-2936130912.html
agora vou aguardar: 4
25883 Buscando item https://www.wimoveis.com.br/propriedades/cobertura-linda!-alto-padrao!toda-planejada-por-2945981965.html
agora vou aguardar: 1
25884 Buscando item https://www.wimoveis.com.br/propriedades/res-natalia-valois-3-quartos-varanda-2948055146.html
agora vou aguardar: 2
25885 

,nome,valor_venda,url,downloaded,condominio,IPTU,lat,long,area_total,area_util,banheiros,vagas,quartos,suites,idade,endereco,bairro,idade_anuncio
0,Sqn 107 Asa Norte Apartamento 4 Quartos 1 Suít...,1390000,https://www.wimoveis.com.br/propriedades/sqn-1...,True,R$ 970,NaN,-15.790497,-47.884679,169m²,133m²,3.0,1.0,4.0,1.0,NaN,"QD Sqn 107 , Asa Norte, Brasília",", Asa Norte, Brasília",mais de 1 ano\n
1,Frente Ao Parque Burle Marx - 4 Súite - Nw 110,2108549,https://www.wimoveis.com.br/propriedades/frent...,True,NaN,NaN,-15.742130,-47.910920,193m²,192m²,5.0,3.0,4.0,4.0,NaN,"Quadra SQNW 110, Noroeste, Brasília",", Noroeste, Brasília",234
2,Lindo Apto 2 Qtos Decorado Ar - Condicionado A...,350000,https://www.wimoveis.com.br/propriedades/lindo...,True,NaN,NaN,-15.835330,-48.013984,51m²,51m²,1.0,1.0,2.0,0.0,5,"Rua 07 norte, Aguas Norte, Águas Claras",", Aguas Norte, Águas Claras",14
3,"Apartamento à Venda no Guará, 2 Quartos - Resi...",347000,https://www.wimoveis.com.br/propriedades/apart...,True,NaN,NaN,-15.850292,-47.971731,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sqn 105 Bloco J 3º Andar, Permuta Por Apto 3q ...",865000,https://www.wimoveis.com.br/propriedades/sqn-1...,True,R$ 700,R$ 1.330,-15.773160,-47.882342,125m²,125m²,3.0,0.0,3.0,1.0,37,"SQN 105 Bloco J, Asa Norte, Asa Norte, Brasília",", Asa Norte, Brasília",71
5,"Excelente Apartamento Cobertura, 223 m², Vazad...",1750000,https://www.wimoveis.com.br/propriedades/excel...,True,NaN,NaN,-15.755861,-47.892336,223m²,223m²,3.0,2.0,4.0,2.0,NaN,"SQN 311 BLOCO L, Asa Norte, Brasília",", Asa Norte, Brasília",26
6,Apartamento 3 Quartos Com Suíte Reformado Res ...,430000,https://www.wimoveis.com.br/propriedades/apart...,True,R$ 530,R$ 740,-15.839408,-48.020902,96m²,76m²,3.0,1.0,3.0,1.0,10,"Rua 16 sul, Aguas Sul, Águas Claras",", Aguas Sul, Águas Claras",\n Publicado hoje\n
7,"Apartamento no Setor Progresso, Goiânia - Goiá...",152900,https://www.wimoveis.com.br/propriedades/apart...,True,NaN,NaN,-16.638326,-49.284353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Central Park 3 Qtos 78 m² Reformado Armarios L...,445000,https://www.wimoveis.com.br/propriedades/centr...,True,R$ 470,NaN,-15.842340,-48.027661,78m²,78m²,3.0,1.0,3.0,1.0,6,"av pau brasil, Centro, Águas Claras",", Centro, Águas Claras",45
9,Cnb 8 Ed Jamaica - 2 Quartos,180000,https://www.wimoveis.com.br/propriedades/cnb-8...,True,R$ 470,NaN,-15.825845,-48.060618,78m²,78m²,2.0,0.0,2.0,0.0,NaN,"QD Cnb 8 , Taguatinga Norte, Taguatinga",", Taguatinga Norte, Taguatinga",mais de 1 ano\n


In [6]:
df.to_csv(f"data/{tipo_imovel}_completo.csv", index=False)

In [7]:
df.isna().sum()

nome                 0
valor_venda          0
url                  0
downloaded           0
condominio       11826
IPTU             19392
lat               3002
long              3002
area_total        8690
area_util         4580
banheiros         4816
vagas             4148
quartos           4136
suites            4212
idade             9412
endereco          4134
bairro            4134
idade_anuncio     4134
dtype: int64